In [1]:
import os, numpy as np, pandas as pd, datetime as dt, pathlib
from dotenv import load_dotenv
from sklearn.preprocessing import MinMaxScaler
from src.cleaning_new import *

path = r"C:\Users\Kartavya\bootcamp_Kartavya_Joshi\homework\homework\homework6"
os.chdir(path)
os.getcwd()

'C:\\Users\\Kartavya\\bootcamp_Kartavya_Joshi\\homework\\homework\\homework6'

In [2]:
RAW_DIR=pathlib.Path((os.getenv("DATA_DIR_RAW","data/raw")))
PROCESS_DIR=pathlib.Path(os.getenv("PROCESS_DIR_RAW","data/processed"))
NOTE_DIR=pathlib.Path(os.getenv("NOTE_DIR_RAW","notebook"))
SRC_DIR=pathlib.Path(os.getenv("SRC_DIR_RAW","src"))
RAW_DIR.mkdir(parents=True, exist_ok=True)
print("RAW_DIR:",RAW_DIR.resolve())
PROCESS_DIR.mkdir(parents=True, exist_ok=True)
print("PROCESS_DIR:",PROCESS_DIR.resolve())
NOTE_DIR.mkdir(parents=True, exist_ok=True)
print("RAW_DIR:",NOTE_DIR.resolve())
SRC_DIR.mkdir(parents=True, exist_ok=True)
print("RAW_DIR:",SRC_DIR.resolve())

RAW_DIR: C:\Users\Kartavya\bootcamp_Kartavya_Joshi\homework\homework\homework6\data\raw
PROCESS_DIR: C:\Users\Kartavya\bootcamp_Kartavya_Joshi\homework\homework\homework6\data\processed
RAW_DIR: C:\Users\Kartavya\bootcamp_Kartavya_Joshi\homework\homework\homework6\notebook
RAW_DIR: C:\Users\Kartavya\bootcamp_Kartavya_Joshi\homework\homework\homework6\src


Create dataset

In [4]:
csv_path = 'data/raw/data_raw_create.csv'
os.makedirs('data', exist_ok=True)

if not os.path.exists(csv_path):
    df_demo = pd.DataFrame({
        'numeric_col': [10, None, 40, 55, 70],
        'category_col': ['A', 'B', 'A', 'B', 'C'],
        'price': ['$100', '$200', '$150', None, '$250'],
        'date_str': ['2025-08-01','2025-08-02',None,'2025-08-04','2025-08-05'],
        'category': ['Electronics','Furniture','Toys','Clothing',None]
    })
    df_demo.to_csv(csv_path, index=False)
    print(f"Demo CSV created at {csv_path}")
else:
    print(f"CSV already exists at {csv_path}")



CSV already exists at data/raw/data_raw_create.csv


Load data

In [6]:
df = pd.read_csv(csv_path)

df_processed= df
df.head()

,numeric_col,category_col,price,date_str,category
0,10.0,A,$100,2025-08-01,Electronics
1,NaN,B,$200,2025-08-02,Furniture
2,40.0,A,$150,NaN,Toys
3,55.0,B,NaN,2025-08-04,Clothing
4,70.0,C,$250,2025-08-05,NaN


In [7]:
# Forward fill numeric_col NA values
df_processed['numeric_col']= df_processed['numeric_col'].ffill()
df_processed

,numeric_col,category_col,price,date_str,category
0,10.0,A,$100,2025-08-01,Electronics
1,10.0,B,$200,2025-08-02,Furniture
2,40.0,A,$150,NaN,Toys
3,55.0,B,NaN,2025-08-04,Clothing
4,70.0,C,$250,2025-08-05,NaN


In [8]:
# Forward fill price NA values
df_processed['price']= df_processed['price'].ffill()
df_processed

,numeric_col,category_col,price,date_str,category
0,10.0,A,$100,2025-08-01,Electronics
1,10.0,B,$200,2025-08-02,Furniture
2,40.0,A,$150,NaN,Toys
3,55.0,B,$150,2025-08-04,Clothing
4,70.0,C,$250,2025-08-05,NaN


In [9]:
# Convert float to str

if 'price' in df.columns:
    df_processed['price'] = df_processed['price'].str.replace('$','').astype(float)

# Convert string to datetime
df_processed["date_str"] = pd.to_datetime(df_processed["date_str"], errors="coerce")




In [10]:
# Applying missing_median
df_processed=missing_median_(df_processed)
df_processed

,numeric_col,category_col,price,date_str,category
0,10.0,A,100.0,2025-08-01,Electronics
1,10.0,B,200.0,2025-08-02,Furniture
2,40.0,A,150.0,NaT,Toys
3,55.0,B,150.0,2025-08-04,Clothing
4,70.0,C,250.0,2025-08-05,NaN


In [11]:
#Applying drop_missing
df_processed=drop_missing(df)
df_processed

,numeric_col,category_col,price,date_str,category
0,10.0,A,100.0,2025-08-01,Electronics
1,10.0,B,200.0,2025-08-02,Furniture
3,55.0,B,150.0,2025-08-04,Clothing


In [12]:
# Normalize_data
normalize_data(df_processed, method='minmax')

,numeric_col,category_col,price,date_str,category
0,0.0,A,0.0,2025-08-01,Electronics
1,0.0,B,1.0,2025-08-02,Furniture
3,1.0,B,0.5,2025-08-04,Clothing


In [13]:
csv_path1 = 'data/processed/data_cleaned.csv'
df_processed.to_csv(csv_path1, index=False)
print("data_cleaned.csv saved")

data_cleaned.csv saved


In [14]:
compare_raw_vs_processed(df, df_processed)

Shape Comparison:
Raw shape:       (5, 5)
Processed shape: (3, 5)

 Missing Values (Raw):
numeric_col     0
category_col    0
price           0
date_str        1
category        1
dtype: int64

 Missing Values (Processed):
numeric_col     0
category_col    0
price           0
date_str        0
category        0
dtype: int64

Dtypes Comparison:
                   raw_dtype processed_dtype
numeric_col          float64         float64
category_col          object          object
price                float64         float64
date_str      datetime64[ns]  datetime64[ns]
category              object          object

